# 真实值评估的示例，北京旅游

https://github.com/truera/trulens/blob/main/trulens_eval/examples/expositional/frameworks/llama_index/llama_index_groundtruth.ipynb

初步结论：

- 可以跑通api
- 没有得到想要的结果，`ground truth` 等指标

In [1]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [2]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.chunk_size = 128
Settings.chunk_overlap = 16

Settings.llm = OpenAILike(
    model="qwen2", 
    api_base="http://monkey:11434/v1", 
    api_key="ollama",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model =OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://monkey:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 27.8 ms, sys: 0 ns, total: 27.8 ms
Wall time: 27.2 ms


In [3]:
%%time

from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://baike.baidu.com/item/%E9%A2%90%E5%92%8C%E5%9B%AD/63458"]
)
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

CPU times: user 2.52 s, sys: 92.4 ms, total: 2.61 s
Wall time: 48.8 s


In [4]:
golden_set = [
    {
        "query": "颐和园在哪里？",
        "response": "颐和园位于北京市西郊海淀区新建宫门路19号。",
    },
]

In [5]:
import os
from trulens_eval.feedback.provider import OpenAI

os.environ["OPENAI_API_KEY"] = "sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750"
os.environ["OPENAI_API_BASE"] = "https://ape:3000/v1"
provider = OpenAI(
    model_engine="gpt-4-turbo"
)

In [6]:
from trulens_eval import Feedback
from trulens_eval.feedback import GroundTruthAgreement
from trulens_eval.app import App

f_groundtruth = Feedback(
    GroundTruthAgreement(golden_set).agreement_measure, name="Ground Truth Eval"
).on_input_output()

✅ In Ground Truth Eval, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Ground Truth Eval, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [7]:
from trulens_eval import TruLlama

tru_query_engine_recorder = TruLlama(
    query_engine,
    app_id="LlamaIndex_App1",
    feedbacks=[f_groundtruth],
)

In [8]:
import nltk
nltk.set_proxy('http://myproxy:7890')

# Run and evaluate on groundtruth questions
for pair in golden_set:
    with tru_query_engine_recorder as recording:
        llm_response = query_engine.query(pair["query"])
        print(llm_response)

颐和园位于北京市西郊海淀区新建宫门路19号。


In [9]:
records, feedback = tru.get_records_and_feedback(
    app_ids=[]
)  # pass an empty list of app_ids to get all
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,latency,total_tokens,total_cost
0,LlamaIndex_App1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_4d1204bf76a84aeac8fa04814c8aef8c,"""\u9890\u548c\u56ed\u5728\u54ea\u91cc\uff1f""","""\u9890\u548c\u56ed\u4f4d\u4e8e\u5317\u4eac\u5...",-,"{""record_id"": ""record_hash_4d1204bf76a84aeac8f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-06T11:17:54.924637"", ""...",2024-08-06T11:17:58.804649,3,358,0.0


In [10]:
tru_query_engine_recorder.app_id

'LlamaIndex_App1'

In [11]:
tru.get_leaderboard(app_ids=[tru_query_engine_recorder.app_id])

,latency,total_cost
app_id,,
LlamaIndex_App1,3.0,0.0
